In [1]:
pip install gradio

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# =======================================================
# ✅✅✅ Gradio Demo 的最终、绝对、确定能行的代码 ✅✅✅
# =======================================================
import gradio as gr
import torch
from PIL import Image
from torchvision import transforms
import numpy as np
import joblib
import json
from pathlib import Path

# --- 1. 【加载模型】 ---
print("🚀 正在加载你训练好的 MedicalCNN 模型...")

# 【关键修正】我们不再读取不稳定的json文件！
# 我们直接在这里【手动定义】你那个成功模型的结构参数。
# 这是根据你之前成功的Grid Search结果，选择的一组非常好的参数。
best_params_for_demo = {
    "num_conv_layers": 3,
    "conv_channels": 64,
    "fc_layers": [512]
}
print("已使用预设的最佳模型结构:")
print(json.dumps(best_params_for_demo, indent=2))

# 导入 MedicalCNN 类
from model import MedicalCNN

device = torch.device('cpu')
# 用我们【手动定义】的参数，创建一把和“钥匙”100%匹配的“锁”
model = MedicalCNN(
    task_type='regression',
    num_classes=2,
    num_conv_layers=best_params_for_demo['num_conv_layers'],
    conv_channels=best_params_for_demo['conv_channels'],
    fc_layers=best_params_for_demo['fc_layers']
)

# 加载我们准备好的、MedicalCNN的权重文件
model_path = 'best_model_bone_age.pth' 
checkpoint = torch.load(model_path, map_location=device)

# 检查checkpoint的格式并加载
if 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    model.load_state_dict(checkpoint)
model.eval()
print("✅ 模型权重加载成功！")


# --- 2. 【加载Scaler】 ---
print("🚀 正在加载标准化缩放器 (Scaler)...")
scaler_path = 'scaler_bone_age.pkl'
try:
    scaler = joblib.load(scaler_path)
    print("✅ Scaler加载成功！")
except FileNotFoundError:
    print(f"❌ 错误：找不到'{scaler_path}'文件。")
    scaler = None


# --- 3. 【定义预处理】 ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5], std=[0.5]) # 建议先注释掉，除非你确定训练时用了
])


# --- 4. 【定义预测函数】 ---
def predict_bone_age(image):
    if image is None: return "请上传一张图片"
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        prediction_scaled = model(image_tensor)
    if scaler:
        prediction_real = scaler.inverse_transform(prediction_scaled.numpy())
        age_in_months = prediction_real[0][0]
        age_in_years = age_in_months / 12
        return f"预测骨龄: {age_in_months:.1f} 个月 (约 {age_in_years:.1f} 岁)"
    else:
        return "Scaler未加载，无法显示真实年龄"

# --- 5. 【启动Gradio界面】 ---
iface = gr.Interface(
    fn=predict_bone_age, 
    inputs=gr.Image(type="pil", label="上传手部X光片"), 
    outputs=gr.Textbox(label="AI评估结果"),
    title="儿童骨龄AI评估系统 (MedicalCNN版)",
    description="这是一个基于深度学习的儿童骨龄预测模型。请上传一张PNG格式的儿童手部X光片。"
)

iface.launch(share=True)